In [3]:
%load_ext autoreload
%autoreload 2


##################################
# INPUT PARAMETERS:
##################################
sat_file = 'starlette'
arc = '030914_2wk'
grav_id ='goco05s' 
local_path = '/data/analysis/notebooks/starlette_analysis/'


##################################
# PATH TO DENSITY MODEL RUN of Choice:
##################################

msis86_model = 'msis86'
path_to_msis86 = '/data/runs_geodyn/st/results/'+ grav_id+'_'+msis86_model+'/'

import sys  
sys.path.insert(0, '/data/analysis/notebooks/util_funcs/py_starlette/')
SAT_ID = 7501001
sat = 'st'
file_name =  sat + arc + '.'+ grav_id
print('The base file name for this arc is:',file_name)
import os

path_to_data = path_to_msis86
iieout_file  = path_to_data + 'IIEOUT/'+ file_name

if os.path.isfile(iieout_file) == True:
    print('File exists: iieout',iieout_file)
else:
    print('Not the correct path for file: iieout' )



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The base file name for this arc is: st030914_2wk.goco05s
File exists: iieout /data/runs_geodyn/st/results/goco05s_msis86/IIEOUT/st030914_2wk.goco05s


In [115]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
import os.path
import linecache



# find the satellites in the GEODYN Run:

# def find_Sat_IDs_resids(iieout_file):
#     text="STATION-SATELLITE CONFIGURATION  DSS1WRNG           9806701"
#     allsats = []
#     with open(iieout_file, 'r') as f:

#         for line_no, line in enumerate(f):
#             if text in line:
#                 # print(line[90:100])
#                 allsats.append(int(line[70:81]) )
#                 # print(line)

#     SatIDs = []
#     for sat in allsats:
#         if sat not in SatIDs:
#             SatIDs.append(sat)
#     iteration = iteration_number(iieout_file)
#     text_obs_resid = 'OBSERVATION RESIDUALS FOR ARC  1 FOR INNER ITERATION  '+ str(int(iteration))

#     lines_list = [] #np.empty(np.size(num_observations))
#     with open(iieout_file, 'r') as f:

#         for line_no, line in enumerate(f):
#             if text_obs_resid in line:
#                     # print(line_no)
#                 lines_list.append(line_no)
#     line_no_1 = lines_list[0]
#     line_no_2 = lines_list[-1]

#     # find the satellites in the GEODYN Run:

#     text="STATION-SATELLITE CONFIGURATION  DSS1WRNG           9806701"
#     allsats = []
#     with open(iieout_file, 'r') as f:

#         for line_no, line in enumerate(f):
#             if text in line:
#           # print(line[90:100])
#                 allsats.append(int(line[70:81]) )
#           # print(line)

#     #         line_no_1 =lines_list 

#     SatIDs = []
#     for sat in allsats:
#         if sat not in SatIDs:
#             SatIDs.append(sat)

#     return SatIDs


def make_datetime_column_resids(resid_df):
    VERBOSE_timer = True
    if VERBOSE_timer == True:
        import time
        start = time.time()
    else:
        pass

    timeHHMM = [] 
    for i,val in enumerate(resid_df['HHMM']):
        if len(val) == 3:
            timehhmm_val = '0'+ val
            timeHHMM.append(timehhmm_val)
        if len(val) == 2:
            timehhmm_val = '00'+ val
            timeHHMM.append(timehhmm_val)
        if len(val) == 4:
            timehhmm_val = val
            timeHHMM.append(timehhmm_val)
        if len(val) == 1:
            timehhmm_val = '000'+ val
            timeHHMM.append(timehhmm_val)

    np.shape(timeHHMM)
    resid_df['timeHHMM'] = timeHHMM

    year  = []
    month = []
    day   = []
    hours  = []
    minutes = []
    secs  = []
    microsecs = []
    for i,val in enumerate(resid_df['YYMMDD']):
        print(val[:2])
        year.append('200' + val[:1])
        month.append(val[1:3])
        day.append(val[3:])

        hours.append(resid_df['timeHHMM'][i][:2])
        minutes.append(resid_df['timeHHMM'][i][2:4])

        secs.append(resid_df['Sec-UTC-R'][i][:2])
        microsecs.append(resid_df['Sec-UTC-R'][i][3:])

    resid_df['year']  = year
    resid_df['month'] = month
    resid_df['day']   = day
    resid_df['hours']  = hours
    resid_df['minutes'] = minutes
    resid_df['secs']  = secs
    resid_df['microsecs'] = microsecs
    if VERBOSE_timer == True:
        end = time.time()
        elapsed = end - start
        print("Loop through and extract indiv date vals:",elapsed)
    else:
        pass

    fix_decimal = []
    for i,val in enumerate(resid_df['secs'].astype(str)):
    #     print(i,val)
        if val.find('.') == 1:
    #             print(i, val)
            fix_decimal.append( '0'+val[:-1])
    #             print(newval)
        else:
            fix_decimal.append( val)

    if VERBOSE_timer == True:
        end = time.time()
        elapsed = end - start
        print("Fix decimals in the seconds column:",elapsed)
    else:
        pass

    year= list(map(int, resid_df['year'].values))
    month= list(map(int, resid_df['month'].values))
    day= list(map(int, resid_df['day'].values))
    hour= list(map(int, resid_df['hours'].values))
    minute = list(map(int, resid_df['minutes'].values))
    second = list(map(int, fix_decimal))
    microsecond= list(map(int, resid_df['microsecs'].values))

    DATE = list(map(datetime, year,month, day, hour,minute,second,microsecond ))

    if VERBOSE_timer == True:
        end = time.time()
        elapsed = end - start
        print("Put all dates in a single column:",elapsed)
    else:
        pass


    return(DATE)

# def read_observed_resids_by_sat(iieout_file ):
#     VERBOSE_timer =True
#     # def read_obs_residuals(iieout_file, iteration, VERBOSE_timer):
#     iteration = str(int(iteration_number(iieout_file)))
#     # VERBOSE_timer = True


#     if VERBOSE_timer == True:
#         import time
#         start = time.time()
#     else:
#         pass

#     #-------------------------------------------------------------------------------

#     #-------------------------------------------------------------------------------
#     #-------------------------------------------------------------------------------
#     '''
#       Now find all the instances of the OBSERVATION RESIDUALS 
#       header at this iteration.  These are stored in a list.
#     '''

#     text_obs_resid = 'OBSERVATION RESIDUALS FOR ARC  1 FOR INNER ITERATION  '+ str(int(iteration))
#     SatIDs = find_Sat_IDs_resids(iieout_file)

#     lines_list = [] #np.empty(np.size(num_observations))
#     with open(iieout_file, 'r') as f:

#         for line_no, line in enumerate(f):
#             if text_obs_resid in line:
#     #                 print(line_no)
#                 lines_list.append(line_no)
#     #         line_no_1 =lines_list 

#     import time
#     start = time.time()

#     init_df = pd.DataFrame(data={'YYMMDD'   :[],
#                             'HHMM'    :[],
#                             'Sec-UTC-R' :[],
#                             'Observation'       :[],
#                             'Residual'       :[],
#                             'Ratio to sigma'       :[],
#                             'Elev1'    :[],
#                             'Elev2'    :[],
#                             'OBS No.'    :[],
#                             'Block'     :[],})

#     dict_sat = {}
#     for i in SatIDs:
#         dict_sat[i]= init_df


#     print(lines_list)
#     for i,iline in enumerate(lines_list):

#         with open(iieout_file, 'r') as f: 
#             for _ in range(iline+1): 
#                 f.readline() 
#             line = f.readline()
# #             print(line[54:62])
#             print(line)
#             print(sat_line)

#         RESID_OBSERV = pd.read_csv(iieout_file, 
#                                           skiprows = lines_list[0] + 6 , 
#                                           nrows =  int((lines_list[0 + 1]-6 - lines_list[0]-7) ),
#                                 names = ['YYMMDD',
#                                         'HHMM',
#                                         'Sec-UTC-R',
#                                         'Observation',
#                                           'Residual',
#                                         'Ratio to sigma',
#                                         'Elev1',
#                                         'Elev2',
#                                         'OBS No.',
#                                         'Block'],
#                                 sep = '\s+',
#                                 )

#     A = pd.DataFrame(RESID_OBSERV)
#     B = dict_sat[sat_line]

#     dict_sat[sat_line] =  pd.concat([ B, A])
#     end = time.time()
#     elapsed = end - start
#     print("Elapsed time:",elapsed)

#     print(i ,'/', str(len(lines_list)))
#     return RESID_OBSERV


# # 

In [116]:
def read_observed_resids_all(iieout_file):

    iteration = str(int(iteration_number(iieout_file)))
    VERBOSE_timer = True



    if VERBOSE_timer == True:
        import time
        start = time.time()
    else:
        pass

    #-------------------------------------------------------------------------------
    '''
    First we need to find how many observations there are:
    '''
    text_smry_meas = 'RESIDUAL SUMMARY BY MEASUREMENT TYPE FOR ARC  1 INNER ITERATION  '+ (iteration) +' OF GLOBAL ITERATION 1'
    with open(iieout_file, 'r') as f:
        for line_no, line_text in enumerate(f):
            if text_smry_meas in line_text:
                text_smry_meas_line_no = line_no         
    text_smry_meas_line = int(linecache.getline(iieout_file,text_smry_meas_line_no+4)[54:63])

    num_observations = np.float(text_smry_meas_line)
    text_obs_resid = 'OBSERVATION RESIDUALS FOR ARC  1 FOR INNER ITERATION  '+ (iteration)
    end_of_section = 'RESIDUAL SUMMARY BY STATION AND TYPE FOR ARC  1 INNER ITERATION  '+ (iteration)

    #-------------------------------------------------------------------------------
    #-------------------------------------------------------------------------------
    '''
    Now find all the instances of the OBSERVATION RESIDUALS 
    header at this iteration.  These are stored in a list.
    '''
    lines_list_1 = [] #np.empty(np.size(num_observations))
    lines_list_2 = [] #np.empty(np.size(num_observations))

    with open(iieout_file, 'r') as f:

        for line_no, line in enumerate(f):
            if text_obs_resid in line:
    #                 print(line_no)
                lines_list_1.append(line_no)
    #         line_no_1 =lines_list 
            elif end_of_section in line:
                lines_list_2.append(line_no)


    # lines = search_iiesout_all_line_numbers(iieout_file, text)
    line_no_1 = lines_list_1[0]
    line_no_2 = lines_list_2[-1]
    #-------------------------------------------------------------------------------
    #-------------------------------------------------------------------------------
    '''
    Use the first and last line numbers from above to select
    the sections of data that contains the observation residuals.
    The outputted csv data are stored as A
    '''
    RESID_OBSERV = pd.read_csv(iieout_file, 
                                      skiprows = line_no_1 + 6 , 
                                        nrows =  int((line_no_2 - line_no_1) ),
                            names = ['YYMMDD',
                                    'HHMM',
                                    'Sec-UTC-R',
                                    'Observation',
                                      'Residual',
                                    'Ratio to sigma',
                                    'Elev1',
                                    'Elev2',
                                    'OBS No.',
                                    'Block'],
                            sep = '\s+',
                            )

    if VERBOSE_timer == True:
        end = time.time()
        elapsed = end - start
        print("Elapsed time after line search setup:",elapsed)
    else:
        pass
    #-------------------------------------------------------------------------------
    #-------------------------------------------------------------------------------
    ''' 
      We now need to fix the data that contains a lot of misplaced
      characters, random strings, and headers
    '''
    A = pd.DataFrame(RESID_OBSERV)  # input
    index_list = []
    for index, row in A.iterrows():
        try:
            float(row['OBS No.'])
            float(row['HHMM'])
        except:
            index_list.append(index)
            continue

    B=A.drop(index_list)
    if VERBOSE_timer == True:
        end = time.time()
        elapsed = end - start
        print("Elapsed time after loop through then drop indecies:",elapsed)
    else:
        pass

    #-------------------------------------------------------------------------------
    #-------------------------------------------------------------------------------
    ''' 
      We locate the index of the last observation number and remove 
      all datapoints that are after it in the DataFrame
    '''
    C = B.reset_index()
    index_drops = np.arange(C[C['OBS No.']==str(int(num_observations))].index[0]+1,  C.index.stop)
    index_drops

    D = C.drop(index_drops)
    if VERBOSE_timer == True:
        end = time.time()
        elapsed = end - start
        print("Elapsed time after dropping all bad indicies after last obs no.:",elapsed)

    dates = make_datetime_column_resids(D)
    D['Date'] = dates

    fix_string = []
    for i,val in enumerate(D['Ratio to sigma']):
        try:
            float(val)
            fix_string.append(val)
        except:
            # print(i, val)
            fix_string.append(val[:-1])

    D['Ratio_to_sigma_fixed'] = fix_string
    return D








Elapsed time after line search setup: 1.7377102375030518
Elapsed time after loop through then drop indecies: 2.36500883102417
Elapsed time after dropping all bad indicies after last obs no.: 2.368178129196167
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30

30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
3

In [118]:
D

,index,YYMMDD,HHMM,Sec-UTC-R,Observation,Residual,Ratio to sigma,Elev1,Elev2,OBS No.,...,timeHHMM,year,month,day,hours,minutes,secs,microsecs,Date,Ratio_to_sigma_fixed
0,0,30914,105,55.208831,933.052599576,-0.001484,-0.0015,58.485935,58.485839,1,...,0105,2003,09,14,01,05,55,208831,2003-09-14 01:05:55.208831,-0.0015
1,1,30914,106,15.009306,1005.251863666,-0.000437,-0.0004,51.260530,51.260437,2,...,0106,2003,09,14,01,06,15,009306,2003-09-14 01:06:15.009306,-0.0004
2,2,30914,106,47.010250,1145.739853454,-0.002657,-0.0027,41.550447,41.550361,3,...,0106,2003,09,14,01,06,47,010250,2003-09-14 01:06:47.010250,-0.0027
3,3,30914,107,13.811144,1279.798448021,-0.006932,-0.0069,35.007633,35.007553,4,...,0107,2003,09,14,01,07,13,811144,2003-09-14 01:07:13.811144,-0.0069
4,4,30914,107,47.212357,1461.073729804,-0.011274,-0.0113,28.408244,28.408171,5,...,0107,2003,09,14,01,07,47,212357,2003-09-14 01:07:47.212357,-0.0113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999,5688,30927,2306,16.259539,1260.651605954,0.006996,0.0070,53.959755,53.959671,4000,...,2306,2003,09,27,23,06,16,259539,2003-09-27 23:06:16.259539,0.0070
4000,5689,30927,2306,46.014220,1374.565632664,0.002611,0.0026,46.639968,46.639884,4001,...,2306,2003,09,27,23,06,46,014220,2003-09-27 23:06:46.014220,0.0026
4001,5690,30927,2307,14.118743,1496.430957455,-0.001994,-0.0020,40.669100,40.669018,4002,...,2307,2003,09,27,23,07,14,118743,2003-09-27 23:07:14.118743,-0.0020
4002,5691,30927,2307,46.624060,1650.422940823,-0.006786,-0.0068,34.755223,34.755145,4003,...,2307,2003,09,27,23,07,46,624060,2003-09-27 23:07:46.624060,-0.0068


In [73]:
A[A['OBS No.']==str(int(num_observations))].index[0]

5692